In [1]:
import numpy as np
import pandas as pd

In [2]:
slide_mes = 0
slide_noise = 0
mem_amount = 5

In [3]:
u_future = np.random.normal(0,2,60)
u = u_future[0 + slide_noise  : (slide_noise + mem_amount)]


In [4]:
c = np.random.normal(0,10,mem_amount)
h = np.ones(mem_amount)
n_moment = np.random.normal(0,1,1)

E = np.zeros(15)
X = np.zeros(15)
Y = np.zeros(15)

In [5]:
def slide_measurements():
    global slide_mes
    slide_mes = slide_mes + 1
    u = u_future[0 +slide_mes  : (slide_mes + mem_amount)]

In [6]:
x = np.add(np.dot(u,h),float(n_moment))

y = np.dot(h,x)
e = y -x 

E = np.append(E, e)
X = np.append(X,x)
Y = np.append(Y,y)
slide_measurements()

In [7]:
full_measurments = np.arange(50)

In [8]:
#Initate parameters and variables for test 
mem_size = 3
time = 0
measure = np.array(full_measurments[time:mem_size]) #Recent recorded measurments -- array size is same as memory size
channels = np.array([1,2,4])
h = np.ones(mem_size) # Estimation vector
X_test = np.zeros(mem_size)
Y_test = np.zeros(mem_size)
E_test = np.zeros(mem_size)


In [9]:
q_test = np.zeros(mem_size)
forgetting_factor = 0.3
p_test = np.zeros(1)
P  = np.zeros(50)

R_x_matrix = np.zeros((mem_size,mem_size))
forgetting_factor_matrix =  np.array([(1 ,0, 0),(0, forgetting_factor, 0),(0, 0, forgetting_factor**2)])

In [10]:
# Update the measurments
def update_measurments():
    global time ,measure
    time = time + 1
    measure = np.array(full_measurments[time:time+3])

In [11]:
#Determine x and append to X

x = np.convolve(channels,measure)
assert(len(x) == len(measure) + len(channels)-1)
# The above not neccesery for debug
x  = np.convolve(channels,measure,"valid")
assert(len(x) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".
x = np.add(x,n_moment)

#X_test array is filled as FILO
X_test= np.insert(X_test,0,x)
X_test= np.delete(X_test,(len(X_test)-1))

In [12]:
#Determine y and e append to Y

y = np.convolve(h,X_test,"valid")
assert(len(y) == 1) #To make any prediction the "amount of measurements" aquired must be at "number of channels".

#X_test array is filled as FILO
Y_test= np.insert(Y_test,0,y)
Y_test= np.delete(Y_test,(len(Y_test)-1))

In [13]:
# Determine the error

# Although the first estimation can be ready after some delay
# the estimations are in real time 
# thus error can be determined as

e = np.subtract(y,measure[2])
assert(len(e) == 1)

#X_test array is filled as FILO
E_test= np.insert(E_test,0,e)
E_test= np.delete(E_test,(len(E_test)-1))

In [29]:
#Loading Q array for A_t representation
q_test = X_test
h = np.zeros(mem_size)      # Solution vector initation
P = np.zeros((mem_size,mem_size))

In [30]:
#Determining p array recursively

def update_p():
    global time,p_test,measure,q_test
    if (time > mem_size):
        p_test = forgetting_factor*p_test + np.matmul(np.transpose(q_test), measure)
    elif(time == mem_size):
        # First meanigful p is determined after num of recorded measurements are equal to num of memory
        p_test =  np.matmul(np.transpose(q_test), measure)

In [31]:
#Update the new p 
update_p()

#Store the p for debug purposes
P_store = np.append(P,p_test)

In [32]:
#Function for determining R_x matrix recursively

def update_R_x():
    global time,measure,R_x_matrix,q_test
    if (time > mem_size):
        R_x_matrix = forgetting_factor*R_x_matrix + np.outer(np.transpose(q_test), q_test)
    elif(time == mem_size):
        # First meanigful R_x is determined after num of recorded measurements are equal to num of memory
        R_x_matrix =  np.outer(np.transpose(q_test), q_test) * forgetting_factor_matrix

In [33]:
# Update R_x matrix recursively
update_R_x()

In [34]:
# Althogh what we need is inverse of R_x and not R_x
# Thankfully there is easier method to update inverse of R_x

def update_P():
    global time,measure,R_x_matrix,q_test,P,k

    #Determing  Kalman Gain Factor
    k = np.dot(P,q_test)/(forgetting_factor+ np.dot(np.dot(np.transpose(q_test),P),np.transpose(q_test)))
    
    if (time > mem_size):
        # Sherman-Morrison formula for updating the inverse of R_x
        P = (forgetting_factor**-1)*P -((forgetting_factor**-1)* np.dot(k,np.transpose(q_test))*P)
    elif(time == mem_size):
        # First meanigful P is determined after num of recorded measurements are equal to num of memory
        P = np.linalg.inv(R_x_matrix)

In [35]:
#Update P (Inverse of R_x)
update_P()

In [37]:
def update_solution():
    global time,h,k,measure

    if (time > mem_size):
        h = h + k*(measure -np.transpose(q_test)*h) 
    elif(time == mem_size):
        pass

In [38]:
# Update the solution
update_solution()